In [1]:
import numpy as np
import xarray as xr

from pyrte_rrtmgp import rrtmgp_gas_optics

ERROR_TOLERANCE = 1e-7

rte_rrtmgp_dir = "../../rrtmgp-data"
clear_sky_example_files = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/inputs"

rfmip = xr.load_dataset(
    f"{clear_sky_example_files}/multiple_input4MIPs_radiation_RFMIP_UColorado-RFMIP-1-2_none.nc"
)
rfmip = rfmip.sel(expt=0)  # only one experiment

kdist = xr.load_dataset(f"{rte_rrtmgp_dir}/rrtmgp-gas-sw-g224.nc")
sw_problem = kdist.gas_optics.load_atmospheric_conditions(rfmip)

sw_problem.sfc_alb_dir = rfmip["surface_albedo"].data
sw_problem.total_solar_irradiance = rfmip["total_solar_irradiance"].data
sw_problem.solar_zenith_angle = rfmip["solar_zenith_angle"].values

solver_flux_up, solver_flux_down = sw_problem.solve()

rsu_reference = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/reference/rsu_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"
rsd_reference = f"{rte_rrtmgp_dir}/examples/rfmip-clear-sky/reference/rsd_Efx_RTE-RRTMGP-181204_rad-irf_r1i1p1f1_gn.nc"

# Compare the results with the reference data
rsu = xr.load_dataset(rsu_reference, decode_cf=False)
ref_flux_up = rsu.isel(expt=0)["rsu"].values

rsd = xr.load_dataset(rsd_reference, decode_cf=False)
ref_flux_down = rsd.isel(expt=0)["rsd"].values

assert np.isclose(solver_flux_up, ref_flux_up, atol=ERROR_TOLERANCE).all()
assert np.isclose(solver_flux_down, ref_flux_down, atol=ERROR_TOLERANCE).all()